In [1]:
import pickle
import gzip
from matplotlib import pyplot as plt
import networkx as nx
from tqdm import tqdm
import numpy as np
import molsysmt as msm
from openmm.app import *
from openmm import *
from openmm.unit import *

In [2]:
with gzip.open(f'tests/energies_0.pkl.gz', 'r') as fff:
    energy_dbs=pickle.load(fff)
        
with gzip.open(f'tests/traj_inh_0.pkl.gz', 'r') as fff:
    traj_inh=pickle.load(fff)

with gzip.open(f'tests/coors_0.pkl.gz', 'r') as fff:
    coors_dbs=pickle.load(fff)

with gzip.open(f'tests/dihed_0.pkl.gz', 'r') as fff:
    dihed_dbs=pickle.load(fff)

In [3]:
min_1 = 4486
min_2 = 575

In [4]:
molsys1 = msm.convert("pdbs/V5.pdb")
new_coordinates = msm.pyunitwizard.quantity(value=coors_dbs[min_1], unit='nm')
msm.set(molsys1, element='atom', coordinates=new_coordinates)

molsys2 = msm.convert("pdbs/V5.pdb")
new_coordinates = msm.pyunitwizard.quantity(value=coors_dbs[min_2], unit='nm')
msm.set(molsys2, element='atom', coordinates=new_coordinates)
molsys2 = msm.structure.least_rmsd_fit(molecular_system=molsys2, reference_molecular_system=molsys1)

In [ ]:
def interpolate_images(pos_min1, pos_min2, num_images):
    images = [pos_min1 + i/(num_images-1)*(pos_min2 - pos_min1) for i in range(num_images)]
    return images

In [ ]:
def calculate_spring_forces(images, k_spring):
    spring_forces = []
    for i in range(1, len(images)-1):  # No modificar extremos
        prev_image = images[i-1]
        next_image = images[i+1]
        current_image = images[i]
        
        tangent = next_image - prev_image
        tangent /= np.linalg.norm(tangent)  # Normaliza el vector tangente

        # Proyecta las fuerzas en componentes paralelas y perpendiculares
        spring_force = k_spring * (np.linalg.norm(next_image - current_image) -
                                   np.linalg.norm(current_image - prev_image)) * tangent
        spring_forces.append(spring_force)
    return spring_forces

In [ ]:
pdb_filename = "pdbs/V5.pdb"
pdb = PDBFile(pdb_filename)
forcefield = ForceField("amber14-all.xml", "amber14/tip3p.xml")
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=NoCutoff,  # Sin tratamiento de fronteras periódicas
    constraints=HBonds         # Restringe enlaces de hidrógeno para un paso de integración más largo
)

In [ ]:
# Crear simulaciones para cada imagen
num_images = 10
positions_min1 = msm.get(molsys1, coordinates=True)  # Conformación mínima 1
positions_min2 = msm.get(molsys2, coordinates=True)  # Conformación mínima 2
images = interpolate_images(positions_min1, positions_min2, num_images)

In [ ]:
simulations = []
for i in range(num_images):
    integrator = VerletIntegrator(1.0*femtoseconds)
    simulation = Simulation(pdb.topology, system, integrator)
    simulation.context.setPositions(msm.pyunitwizard.convert(images[i][0],to_form='openmm.unit'))
    simulations.append(simulation)

In [ ]:
def update_positions_with_forces(positions, forces, step_size=0.001 * nanometer):
    """
    Actualiza las posiciones en base a las fuerzas.
    
    Parámetros:
        positions: numpy.array con las posiciones actuales (en nm).
        forces: numpy.array con las fuerzas actuales (en kJ/mol/nm).
        step_size: Escalamiento del desplazamiento (en nm).
    """
    displacement = step_size * (msm.pyunitwizard.get_value(forces / np.linalg.norm(forces, axis=1)[:, None]))
    return positions + displacement

In [ ]:
def run_neb(simulations, k_spring=100*kilojoule_per_mole/nanometer**2, max_iterations=100):
    for iteration in range(max_iterations):
        # Obtener posiciones actuales de todas las imágenes
        positions = [sim.context.getState(getPositions=True).getPositions(asNumpy=True) for sim in simulations]
        
        # Calcular fuerzas físicas y de resorte
        spring_forces = calculate_spring_forces(positions, k_spring)
        
        for i, sim in enumerate(simulations[1:-1], start=1):  # Excluye los extremos
            state = sim.context.getState(getForces=True)
            physical_forces = state.getForces(asNumpy=True)
            
            # Combinar fuerzas físicas y de resorte
            total_forces = physical_forces + spring_forces[i-1]
            sim.context.setVelocitiesToTemperature(0.0)  # Congela el sistema
            positions[i] = update_positions_with_forces(positions[i], total_forces)
            sim.context.setPositions(positions[i])

        # Criterio de convergencia (opcional)
        max_force = max(np.linalg.norm(f) for f in spring_forces)
        print(max_force)
        if max_force < 1e-3:
            break

In [ ]:
run_neb(simulations)

In [ ]:
energy_dbs[min_1]

In [ ]:
energy_dbs[min_2]

In [ ]:
aux = msm.merge([molsys1,molsys2])
msm.view(aux, standard=True)

In [ ]:
path = interpolate_images(molsys1, molsys2, 10)

In [ ]:
msm.view(msm.merge(path), standard=True)

In [ ]:
def calculate_spring_forces(path, k_spring):

    images=[]
    for sys_aux in path:
        images.append(msm.get(sys_aux, coordinates=True))
        
    spring_forces = []
    for i in range(1, len(images)-1):  # No modificar extremos
        prev_image = images[i-1]
        next_image = images[i+1]
        current_image = images[i]
        
        tangent = next_image - prev_image
        tangent /= np.linalg.norm(tangent)  # Normaliza el vector tangente

        # Proyecta las fuerzas en componentes paralelas y perpendiculares
        spring_force = k_spring * (np.linalg.norm(next_image - current_image) -
                                   np.linalg.norm(current_image - prev_image)) * tangent
        spring_forces.append(spring_force)
        
    return spring_forces

In [ ]:
path

In [ ]:
pesos_transiciones={}
pesos_nodos={}

In [ ]:
for ii in range(len(traj_inh)-1):
    nodo_1 = traj_inh[ii]
    nodo_2 = traj_inh[ii+1]
    if (nodo_1, nodo_2) in pesos_transiciones:
        pesos_transiciones[nodo_1, nodo_2]+=1
    else:
        pesos_transiciones[nodo_1, nodo_2]=1
    if nodo_1 in pesos_nodos:
        pesos_nodos[nodo_1]+=1
    else:
        pesos_nodos[nodo_1]=1

In [ ]:
G = nx.DiGraph()  # Para un grafo dirigido

In [ ]:
# Agregar las aristas desde el diccionario
for nodes, peso in pesos_transiciones.items():
    G.add_edge(nodes[0], nodes[1])

In [ ]:
G[4486]

In [ ]:
np.argmax(pesos_nodos)

In [ ]:
peso_max=0
nodo_max=0
for ii,jj in pesos_nodos.items():
    if jj>peso_max:
        nodo_max=ii
        peso_max=jj

In [ ]:
nodo_max

In [ ]:
min_1 = np.argmin(energy_dbs[1])
energy_dbs[1][min_1]

In [ ]:
min_2 = np.argmin(energy_dbs[2])
energy_dbs[2][min_2]

In [ ]:
dihed_dbs[0][min_0]

In [ ]:
dihed_dbs[1][min_1]

In [ ]:
dihed_dbs[2][min_2]

In [ ]:
# Nombre del archivo PDB del péptido
molsys0 = msm.convert("pdbs/V5.pdb")
molsys1 = msm.convert("pdbs/V5.pdb")

In [ ]:
new_coordinates = msm.pyunitwizard.quantity(value=coors_dbs[min_0], unit='nm')
msm.set(molsys0, element='atom', coordinates=new_coordinates)

new_coordinates = msm.pyunitwizard.quantity(value=coors_dbs[575], unit='nm')
msm.set(molsys1, element='atom', coordinates=new_coordinates)


In [ ]:
molsys01 = msm.structure.least_rmsd_fit(molecular_system=molsys0, reference_molecular_system=molsys1)

In [ ]:
aux = msm.merge([molsys01,molsys1])

In [ ]:
msm.view(aux, standard=True)

In [ ]:
G = nx.DiGraph()  # Para un grafo dirigido

In [ ]:
def es_el_mismo(angs1, angs2):
    if np.max(np.abs(angs1-angs2))<0.1:
        return True
    else:
        return False

In [ ]:
pesos_transiciones={}
pesos_nodos={}

In [ ]:
for ii in range(len(traj_inh)-1):
    nodo_1 = traj_inh[ii]
    nodo_2 = traj_inh[ii+1]
    if (nodo_1, nodo_2) in pesos_transiciones:
        pesos_transiciones[nodo_1, nodo_2]+=1
    else:
        pesos_transiciones[nodo_1, nodo_2]=1
    if nodo_1 in pesos_nodos:
        pesos_nodos[nodo_1]+=1
    else:
        pesos_nodos[nodo_1]=1

In [ ]:
G = nx.DiGraph()  # Para un grafo dirigido

In [ ]:
# Agregar las aristas desde el diccionario
for nodes, peso in pesos_transiciones.items():
    G.add_edge(nodes[0], nodes[1])

In [ ]:
nx.draw(G, with_labels=False, node_color="blue", node_size=0.1)
plt.show()